In [1]:
import pandas as pd
import json
import glob
import unicodedata

def remove_accents(input_str: str) -> str:
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore').decode('utf8')
    return only_ascii

pd.options.display.max_columns = 300

dir_declaraciones = 'datos/dataton/'
dir_fun_contrataciones = 'datos/dataton/'

def json_normalize_robust(js):
    try:
        out = pd.json_normalize(js)
    except:
        out = pd.DataFrame()
    return out

# Lectura datos

## Declaraciones

In [2]:
with open(f'{dir_declaraciones}/declaraciones.json', encoding='UTF-8') as jsonfile:
    file_declaracion = json.load(jsonfile)
    decl = pd.json_normalize(file_declaracion)\
        .rename(columns={'declaracion.situacionPatrimonial.datosGenerales.curp': 'curp'})
decl['institucion_merge'] = decl['metadata.institucion'].str.lower().str.replace('\W', '').apply(remove_accents)
decl.to_csv(f'{dir_declaraciones}/declaraciones_2021.csv', index=False)
decl.to_pickle(f'{dir_declaraciones}/declaraciones_2021.pkl')
decl.head()

<ipython-input-2-1d7a5834c1bb>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  decl['institucion_merge'] = decl['metadata.institucion'].str.lower().str.replace('\W', '').apply(remove_accents)


,id,_id.$oid,metadata.actualizacion,metadata.tipo,metadata.institucion,metadata.declaracionCompleta,metadata.actualizacionConflictoInteres,declaracion.situacionPatrimonial.datosGenerales.nombre,declaracion.situacionPatrimonial.datosGenerales.primerApellido,declaracion.situacionPatrimonial.datosGenerales.segundoApellido,curp,declaracion.situacionPatrimonial.datosGenerales.rfc.rfc,declaracion.situacionPatrimonial.datosGenerales.rfc.homoClave,declaracion.situacionPatrimonial.datosGenerales.correoElectronico.institucional,declaracion.situacionPatrimonial.datosGenerales.correoElectronico.personal,declaracion.situacionPatrimonial.datosGenerales.telefono.casa,declaracion.situacionPatrimonial.datosGenerales.telefono.celularPersonal,declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.clave,declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.valor,declaracion.situacionPatrimonial.datosGenerales.paisNacimiento,declaracion.situacionPatrimonial.datosGenerales.nacionalidad,declaracion.situacionPatrimonial.datosGenerales.aclaracionesObservaciones,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.calle,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.numeroExterior,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.numeroInterior,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.coloniaLocalidad,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.municipioAlcaldia.clave,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.municipioAlcaldia.valor,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.entidadFederativa.clave,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.entidadFederativa.valor,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.codigoPostal,declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad,declaracion.situacionPatrimonial.datosCurricularesDeclarante.aclaracionesObservaciones,declaracion.situacionPatrimonial.datosEmpleoCargoComision.tipoOperacion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelOrdenGobierno,declaracion.situacionPatrimonial.datosEmpleoCargoComision.ambitoPublico,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico,declaracion.situacionPatrimonial.datosEmpleoCargoComision.areaAdscripcion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.empleoCargoComision,declaracion.situacionPatrimonial.datosEmpleoCargoComision.contratadoPorHonorarios,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision,declaracion.situacionPatrimonial.datosEmpleoCargoComision.funcionPrincipal,declaracion.situacionPatrimonial.datosEmpleoCargoComision.fechaTomaPosesion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.telefonoOficina.telefono,declaracion.situacionPatrimonial.datosEmpleoCargoComision.telefonoOficina.extension,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.calle,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.numeroExterior,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.numeroInterior,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.coloniaLocalidad,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.municipioAlcaldia.clave,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.municipioAlcaldia.valor,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.entidadFederativa.clave,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.entidadFederativa.valor,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.codigoPostal,declaracion.situacionPatrimonial.datosEmpleoCargoComision.aclaracionesObservaciones,declaracion.situacionPatrimonial.experienciaLaboral.ninguno,declaracion.situacionPatrimonial.experienciaLaboral.exp

In [175]:
decl.columns[decl.columns.str.contains('', case=False)].tolist()

['id',
 '_id.$oid',
 'metadata.actualizacion',
 'metadata.tipo',
 'metadata.institucion',
 'metadata.declaracionCompleta',
 'metadata.actualizacionConflictoInteres',
 'declaracion.situacionPatrimonial.datosGenerales.nombre',
 'declaracion.situacionPatrimonial.datosGenerales.primerApellido',
 'declaracion.situacionPatrimonial.datosGenerales.segundoApellido',
 'curp',
 'declaracion.situacionPatrimonial.datosGenerales.rfc.rfc',
 'declaracion.situacionPatrimonial.datosGenerales.rfc.homoClave',
 'declaracion.situacionPatrimonial.datosGenerales.correoElectronico.institucional',
 'declaracion.situacionPatrimonial.datosGenerales.correoElectronico.personal',
 'declaracion.situacionPatrimonial.datosGenerales.telefono.casa',
 'declaracion.situacionPatrimonial.datosGenerales.telefono.celularPersonal',
 'declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.clave',
 'declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.valor',
 'declaracion.situacionPa

### Educación de funcionarios

In [72]:
educacion = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad'].apply(pd.json_normalize))\
    .pipe(lambda x: pd.concat([v.assign(curp=k) for k, v in x.to_dict().items()]))\
    .reset_index()
educacion.to_csv(f'{dir_declaraciones}/educacion_declaraciones_2021.csv', index=False)
educacion.to_pickle(f'{dir_declaraciones}/educacion_declaraciones_2021.pkl')
educacion.head()

,index,tipoOperacion,carreraAreaConocimiento,status,documentoObtenido,fechaObtencion,nivel.clave,nivel.valor,institucionEducativa.nombre,institucionEducativa.ubicacion,curp,estatus
0,0,SIN_CAMBIOS,Licenciatura en Comercio Internacional,FINALIZADO,TITULO,2008-08-15T07:17:26.107Z,LIC,LICENCIATURA,Universidad Tecnológica de México,MX,ABAF800309HPURÑT08,NaN
1,0,SIN_CAMBIOS,Licenciatura en Economía,FINALIZADO,TITULO,2001-01-27T03:53:39.906Z,LIC,LICENCIATURA,Universidad Panamericana,MX,ABAG780123HVZYRL05,NaN
2,1,SIN_CAMBIOS,Arte y Diseño,FINALIZADO,CONSTANCIA,2007-10-29T04:35:15.410Z,ESP,ESPECIALIDAD,Massachusetts Institute of Technology,EX,ABAG780123HVZYRL05,NaN
3,2,SIN_CAMBIOS,"Ingeniería, Arquitectura, Urbanismo y afines",FINALIZADO,CONSTANCIA,2013-09-08T12:10:24.126Z,DOC,DOCTORADO,Universidad Panamericana,MX,ABAG780123HVZYRL05,NaN
4,0,SIN_CAMBIOS,Licenciatura en Administración de las Tecnolog...,FINALIZADO,TITULO,2011-05-14T09:19:56.468Z,LIC,LICENCIATURA,Universidad Justo Sierra,MX,ABAM750311HPURCR03,NaN


### Experiencia de funcionarios

In [32]:
experiencia = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.experienciaLaboral.experiencia'].apply(pd.json_normalize))\
    .pipe(lambda x: pd.concat([v.assign(curp=k) for k, v in x.to_dict().items()]))\
    .assign(experiencia=lambda x: pd.to_datetime(x['fechaEgreso']).subtract(pd.to_datetime(x['fechaIngreso'])).dt.days)
experiencia.to_csv(f'{dir_declaraciones}/experiencia_declaraciones_2021.csv', index=False)
experiencia.to_pickle(f'{dir_declaraciones}/experiencia_declaraciones_2021.pkl')
experiencia.head()

,tipoOperacion,nombreEmpresaSociedadAsociacion,rfc,area,puesto,fechaIngreso,fechaEgreso,ubicacion,ambitoSector.clave,ambitoSector.valor,sector.clave,sector.valor,curp,nivelOrdenGobierno,ambitoPublico,nombreEntePublico,areaAdscripcion,empleoCargoComision,funcionPrincipal,experiencia
0,SIN_CAMBIOS,Grupo Financiero Scotiabank Inverlat,GRUP881113AB2,Comercial,Atención a clientes,2000-12-20T07:12:55.598Z,2006-10-30T03:42:00.869Z,MX,PRV,PRIVADO,OTRO,Servicios financieros,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,2139
1,SIN_CAMBIOS,FEMSA,FEMS781119CC3,Comercial,Agente de ventas,2007-08-05T00:36:01.639Z,2010-04-17T09:25:49.967Z,MX,PRV,PRIVADO,OTRO,Holding,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,986
2,SIN_CAMBIOS,Grupo BAL,GRUP751016CX2,Recursos Humanos,Repartidor,2011-03-02T04:19:03.989Z,2014-09-26T23:58:27.462Z,MX,PRV,PRIVADO,OTRO,Holding,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,1304
0,SIN_CAMBIOS,Grupo Televisa,GRUP881111AC1,Gerencia,Gerente Regional,2001-01-29T14:04:42.439Z,2006-11-12T13:42:57.818Z,MX,PRV,PRIVADO,OTRO,Telecomunicaciones,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,2112
1,SIN_CAMBIOS,Grupo Financiero BBVA Bancomer,GRUP861126AC1,Comercial,Gerente,2007-09-19T13:10:57.858Z,2010-11-06T05:31:45.396Z,MX,PRV,PRIVADO,OTRO,Servicios financieros,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,1143


### Dependientes económicos

In [42]:
dependientes = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: 
            x['declaracion.situacionPatrimonial.datosDependienteEconomico.dependienteEconomico']
            .apply(json_normalize_robust)
         )\
    .pipe(lambda x: pd.concat([v.assign(curp_funcionario=k) for k, v in x.to_dict().items()]))
dependientes.to_csv(f'{dir_declaraciones}/dependientes_declaraciones_2021.csv', index=False)
dependientes.to_pickle(f'{dir_declaraciones}/dependientes_declaraciones_2021.pkl')
dependientes.head()

,tipoOperacion,nombre,primerApellido,segundoApellido,fechaNacimiento,extranjero,curp,habitaDomicilioDeclarante,lugarDondeReside,rfc.rfc,rfc.homoClave,parentescoRelacion.clave,parentescoRelacion.valor,actividadLaboral.clave,actividadLaboral.valor,curp_funcionario,rfc,domicilioMexico.calle,domicilioMexico.numeroExterior,domicilioMexico.numeroInterior,domicilioMexico.coloniaLocalidad,domicilioMexico.municipioAlcaldia.clave,domicilioMexico.municipioAlcaldia.valor,domicilioMexico.entidadFederativa.clave,domicilioMexico.entidadFederativa.valor,domicilioMexico.codigoPostal,domicilioExtranjero.calle,domicilioExtranjero.numeroExterior,domicilioExtranjero.numeroInterior,domicilioExtranjero.ciudadLocalidad,domicilioExtranjero.estadoProvincia,domicilioExtranjero.pais,domicilioExtranjero.codigoPostal,actividadLaboralSectorPublico.nivelOrdenGobierno,actividadLaboralSectorPublico.ambitoPublico,actividadLaboralSectorPublico.nombreEntePublico,actividadLaboralSectorPublico.areaAdscripcion,actividadLaboralSectorPublico.empleoCargoComision,actividadLaboralSectorPublico.funcionPrincipal,actividadLaboralSectorPublico.salarioMensualNeto.valor,actividadLaboralSectorPublico.salarioMensualNeto.moneda,actividadLaboralSectorPublico.fechaIngreso,actividadLaboralSectorPrivadoOtro.nombreEmpresaSociedadAsociacion,actividadLaboralSectorPrivadoOtro.rfc,actividadLaboralSectorPrivadoOtro.empleoCargo,actividadLaboralSectorPrivadoOtro.fechaIngreso,actividadLaboralSectorPrivadoOtro.salarioMensualNeto.valor,actividadLaboralSectorPrivadoOtro.salarioMensualNeto.moneda,actividadLaboralSectorPrivadoOtro.proveedorContratistaGobierno,actividadLaboralSectorPrivadoOtro.sector.clave,actividadLaboralSectorPrivadoOtro.sector.valor
0,SIN_CAMBIOS,Emilio,Menchaca,Guerra,2012-08-07,False,MEGE20120807MPUNRL07,True,MEXICO,MEGE20120807,A28,HIJ,HIJO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SIN_CAMBIOS,Jerónimo,Carbajal,Xacon,2011-03-12,False,CAXJ20110312MAGRCR02,True,MEXICO,CAXJ20110312,B60,PRI,PRIMO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIN_CAMBIOS,María Eugenia,Kadar rodriguez,Irizarry,2011-07-06,False,KAIM20110706HPUDZR02,True,MEXICO,KAIM20110706,A44,PRI,PRIMO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SIN_CAMBIOS,Elena,Arias,Razo,2009-00-12,False,ARRE20090012MSNSZN04,True,MEXICO,ARRE20090012,D93,NIN,NINGUNO,NIN,NINGUNO,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SIN_CAMBIOS,Dulce María,Ornelas,Morales,2006-07-23,False,ORMD20060723HVZNRL07,True,MEXICO,ORMD20060723,C77,AHI,AHIJADO(A),NIN,NINGUNO,ABAM750311HPURCR03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Bienes inmuebles

In [103]:
bienesinmuebles = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: 
            x['declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble']
            .apply(json_normalize_robust)
         )\
    .pipe(lambda x: pd.concat([v.assign(curp_funcionario=k) for k, v in x.to_dict().items()]))
bienesinmuebles.to_csv(f'{dir_declaraciones}/bienesinmuebles_declaraciones_2021.csv', index=False)
bienesinmuebles.to_pickle(f'{dir_declaraciones}/bienesinmuebles_declaraciones_2021.pkl')
bienesinmuebles.head()

,curp_funcionario,tipoOperacion,titular,porcentajePropiedad,formaPago,valorAdquisicion,fechaAdquisicion,datoIdentificacion,valorConformeA,tipoInmueble.clave,tipoInmueble.valor,superficieTerreno.valor,superficieTerreno.unidad,superficieConstruccion.valor,superficieConstruccion.unidad,formaAdquisicion.clave,formaAdquisicion.valor,domicilioMexico.calle,domicilioMexico.numeroExterior,domicilioMexico.numeroInterior,domicilioMexico.coloniaLocalidad,domicilioMexico.municipioAlcaldia.clave,domicilioMexico.municipioAlcaldia.valor,domicilioMexico.entidadFederativa.clave,domicilioMexico.entidadFederativa.valor,domicilioMexico.codigoPostal,tipo.clave,tipo.valor,tercero,transmisor,valorAdquisicion.valor,valorAdquisicion.moneda,domicilioExtranjero.calle,domicilioExtranjero.numeroExterior,domicilioExtranjero.numeroInterior,domicilioExtranjero.ciudadLocalidad,domicilioExtranjero.estadoProvincia,domicilioExtranjero.pais,domicilioExtranjero.codigoPostal,motivoBaja.clave,motivoBaja.valor
0,ABAG780123HVZYRL05,SIN_CAMBIOS,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",100.0,CONTADO,81306842.0,2002-05-16T00:19:32.105Z,6138b452-53a5-4363-b081-63f31de9617a,ESCRITURA PÚBLICA,CASA,CASA,204.0,m2,165.0,m2,CPV,COMPRAVENTA,Esteban Arrabal,310,,Independencia,034,San Carlos,28,Tamaulipas,87754,PALC,PALCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABAG780123HVZYRL05,SIN_CAMBIOS,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",100.0,NO APLICA,87403907.0,1971-10-27T05:17:25.677Z,21cd9b72-7242-4ce6-8ab7-9336b89970c1,ESCRITURA PÚBLICA,CASA,CASA,6119.0,m2,5799.0,m2,HRN,HERENCIA,Quinta Conjunto,996,,Emiliano Zapata,014,Rosario,25,Sinaloa,80435,EDIF,EDIFICIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABAG780123HVZYRL05,SIN_CAMBIOS,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",100.0,CONTADO,88960136.0,2009-04-30T20:50:25.040Z,0d268d86-d305-4379-a886-47435a66b387,ESCRITURA PÚBLICA,CASA,CASA,412.0,m2,348.0,m2,CPV,COMPRAVENTA,Menchaca Ramal,1339,,Justo Sierra,009,Francisco I. Madero,05,Coahuila de Zaragoza,25426,PALC,PALCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ABAM881010MDFYVT07,SIN_CAMBIOS,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",100.0,CONTADO,2605031.0,1994-12-16T22:02:55.567Z,cceecd67-38df-47db-996f-005a9fb280b4,ESCRITURA PÚBLICA,CASA,CASA,141.0,m2,141.0,m2,CPV,COMPRAVENTA,Gustavo Vía,1631,,Anahuac,006,San Quintín,02,Baja California,21951,BODG,BODEGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ABBA870314MSNRRN07,SIN_CAMBIOS,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",100.0,CRÉDITO,4937025.0,2001-01-23T17:08:13.348Z,6f990c1a-a1ac-4da5-b432-fea15b76f27d,ESCRITURA PÚBLICA,CASA,CASA,179.0,m2,102.0,m2,CPV,COMPRAVENTA,Saúl Calleja,467,,Ejercito Nacional,168,Tepexco,21,Puebla,72503,BODG,BODEGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
bienesinmuebles.shape

(7473, 41)

In [112]:
bienesinmuebles_evolucion = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .set_index(['curp', 'year'])\
    .pipe(lambda x: 
            x['declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble']
            .apply(json_normalize_robust)
         )\
    .pipe(lambda x: pd.concat([v.assign(curp=k[0], year=k[1]) for k, v in x.to_dict().items()]))

In [133]:
bienesinmuebles_evolucion_x_year = bienesinmuebles_evolucion.rename(columns=lambda x: x.replace('.', '_')).query('formaAdquisicion_valor!=["RIFA O SORTEO", "HERENCIA", "SENTENCIA"]')\
    .groupby(['curp', 'year'], as_index=False)['valorAdquisicion'].sum()\
    .rename(columns={'valorAdquisicion': 'valor_inmuebles'})

bienesinmuebles_evolucion_x_year.to_pickle(f'{dir_declaraciones}/evolucion_patrimonio/bienesinmuebles_evolucion_x_year.pkl')
bienesinmuebles_evolucion_x_year

,curp,year,valor_inmuebles
0,ABAG780123HVZYRL05,2015,170266978.0
1,ABAG780123HVZYRL05,2016,170266978.0
2,ABAG780123HVZYRL05,2017,170266978.0
3,ABAG780123HVZYRL05,2018,170266978.0
4,ABAG780123HVZYRL05,2019,170266978.0
...,...,...,...
26314,ÑAUM890715HAGÑRR09,2017,5844712.0
26315,ÑAUM890715HAGÑRR09,2018,5844712.0
26316,ÑAUM890715HAGÑRR09,2019,5844712.0
26317,ÑAUM890715HAGÑRR09,2020,5844712.0


### Inversiones

In [46]:
inversiones = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.inversiones.inversion'].apply(json_normalize_robust))\
    .pipe(lambda x: pd.concat([v.assign(curp_funcionario=k) for k, v in x.to_dict().items()]))
inversiones.to_csv(f'{dir_declaraciones}/inversiones_declaraciones_2021.csv', index=False)
inversiones.to_pickle(f'{dir_declaraciones}/inversiones_declaraciones_2021.pkl')
inversiones.head()

,tipoOperacion,titular,numeroCuentaContrato,porcentajeIncrementoDecremento,tipoInversion.clave,tipoInversion.valor,subTipoInversion.clave,subTipoInversion.valor,localizacionInversion.pais,localizacionInversion.institucionRazonSocial,localizacionInversion.rfc,saldoFechaConclusion.valor,saldoFechaConclusion.moneda,curp_funcionario,subTipoInversion.tipoInversion
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",87349554,-22%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Valdez, Méndez and Sanabria",68b4640a-5458-47aa-9b8c-e4ea744af3d6,10954.0,MXN,ABAF800309HPURÑT08,NaN
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",21866734,-10%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Padrón, Quintero and Fernández",f444de0b-3b7d-45b4-a6f0-9edcb782f89b,26637502.0,MXN,ABAG780123HVZYRL05,NaN
1,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",32190625,5%,BANC,BANCARIA,CTES,CERTIFICADOS DE LA TESORERÍA,MX,Barajas e Hijos,1de64040-77c8-4e66-93df-495666c4e15d,26958683.0,MXN,ABAG780123HVZYRL05,AFOT
2,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",29245153,-12%,FINV,FONDOS DE INVERSIÓN,CAHO,CUENTA DE AHORRO,MX,Melgar - Zambrana,92c6b093-b455-4c04-8282-c8867e6cb929,5719274.0,MXN,ABAG780123HVZYRL05,BANC
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",22013834,-8%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Pulido, Gallardo and Alaniz",2c22f844-6861-48f9-91b9-189099419bd7,10186.0,MXN,ABAM750311HPURCR03,NaN


In [127]:
inversiones_evolucion = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .set_index(['curp', 'year'])\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.inversiones.inversion'].apply(json_normalize_robust))\
    .pipe(lambda x: pd.concat([v.assign(curp=k[0], year=k[1]) for k, v in x.to_dict().items()]))

In [131]:
inversiones_evolucion_x_year = inversiones_evolucion.groupby(['curp', 'year'], as_index=False)['saldoDiciembreAnterior.valor'].sum()\
    .rename(columns={'saldoDiciembreAnterior.valor': 'valor_inversiones'})

inversiones_evolucion_x_year.to_pickle(f'{dir_declaraciones}/evolucion_patrimonio/inversiones_evolucion_x_year.pkl')
inversiones_evolucion_x_year

,curp,year,valor_inversiones
0,ABAF800309HPURÑT08,2015,0.0
1,ABAF800309HPURÑT08,2016,23111.0
2,ABAF800309HPURÑT08,2017,18488.0
3,ABAF800309HPURÑT08,2018,15529.0
4,ABAF800309HPURÑT08,2019,18479.0
...,...,...,...
34995,ÑAUM890715HAGÑRR09,2017,733828.0
34996,ÑAUM890715HAGÑRR09,2018,706957.0
34997,ÑAUM890715HAGÑRR09,2019,741521.0
34998,ÑAUM890715HAGÑRR09,2020,753992.0


### Vehículos

In [48]:
vehiculos = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .drop_duplicates(subset=['curp'])\
    .set_index('curp')\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.vehiculos.vehiculo'].apply(json_normalize_robust))\
    .pipe(lambda x: pd.concat([v.assign(curp_funcionario=k) for k, v in x.to_dict().items()]))
vehiculos.to_csv(f'{dir_declaraciones}/vehiculos_declaraciones_2021.csv', index=False)
vehiculos.to_pickle(f'{dir_declaraciones}/vehiculos_declaraciones_2021.pkl')
vehiculos.head()

,tipoOperacion,marca,modelo,anio,numeroSerieRegistro,tipoVehiculo.clave,tipoVehiculo.valor,titularBien.clave,titularBien.valor,valorAdquisicion.valor,valorAdquisicion.moneda,curp_funcionario,titular,transmisor,tercero,formaPago,fechaAdquisicion,lugarRegistro.pais,lugarRegistro.entidadFederativa.clave,lugarRegistro.entidadFederativa.valor,formaAdquisicion.clave,formaAdquisicion.valor,motivoBaja.clave,motivoBaja.valor
0,BAJA,opel,manta,1974,75248817-8f2e-4168-bbf6-3bf349f34636,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,123065.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAJA,volvo,264gl,1978,e24fba0f-1c38-4955-9bef-e0d4ada7a9db,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,169142.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAJA,chevrolet,citation,1982,822fa386-d514-4f12-a211-ed47bc9b747b,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,40446.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAJA,datsun,pl510,1971,555e9710-49d5-4e67-a665-07659f62db28,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,147112.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BAJA,ford,granada ghia,1976,cca40092-3cca-44ff-9fbe-5651277434c6,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,125782.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
vehiculos_evolucion = decl.sort_values(['curp', 'metadata.actualizacion'], ascending=[True, False])\
    .set_index(['curp', 'year'])\
    .pipe(lambda x: x['declaracion.situacionPatrimonial.vehiculos.vehiculo'].apply(json_normalize_robust))\
    .pipe(lambda x: pd.concat([v.assign(curp=k[0], year=k[1]) for k, v in x.to_dict().items()]))

In [132]:
vehiculos_evolucion_x_year = vehiculos_evolucion.groupby(['curp', 'year'], as_index=False)['valorAdquisicion.valor'].sum()\
    .rename(columns={'valorAdquisicion.valor': 'valor_vehiculos'})

vehiculos_evolucion_x_year.to_pickle(f'{dir_declaraciones}/evolucion_patrimonio/vehiculos_evolucion_x_year.pkl')
vehiculos_evolucion_x_year

,curp,year,valor_vehiculos
0,ABAF800309HPURÑT08,2015,123065.0
1,ABAF800309HPURÑT08,2016,332653.0
2,ABAF800309HPURÑT08,2017,479765.0
3,ABAF800309HPURÑT08,2018,605547.0
4,ABAF800309HPURÑT08,2019,695333.0
...,...,...,...
29248,ÑAUM890715HAGÑRR09,2017,1548106.0
29249,ÑAUM890715HAGÑRR09,2018,1932839.0
29250,ÑAUM890715HAGÑRR09,2019,2490573.0
29251,ÑAUM890715HAGÑRR09,2020,2490573.0


### Evolución del patrimonio

In [ ]:
vehiculos_evolucion_x_year = pd.read_pickle(f'{dir_declaraciones}/evolucion_patrimonio/vehiculos_evolucion_x_year.pkl')
inversiones_evolucion_x_year = pd.read_pickle(f'{dir_declaraciones}/evolucion_patrimonio/inversiones_evolucion_x_year.pkl')
bienesinmuebles_evolucion_x_year = pd.read_pickle(f'{dir_declaraciones}/evolucion_patrimonio/bienesinmuebles_evolucion_x_year.pkl')

In [143]:
patrimonio_total_evolucion = decl[['curp', 'year']].drop_duplicates().reset_index(drop=True)\
    .merge(vehiculos_evolucion_x_year, on=['curp', 'year'], how='left')\
    .merge(bienesinmuebles_evolucion_x_year, on=['curp', 'year'], how='left')\
    .merge(inversiones_evolucion_x_year, on=['curp', 'year'], how='left')\
    .fillna(0)\
    .assign(patrimonio_total=lambda x: x['valor_vehiculos'] + x['valor_inmuebles'] + x['valor_inversiones'])

patrimonio_total_evolucion.to_csv(f'{dir_declaraciones}/evolucion_patrimonio/patrimonio_total_evolucion.csv', index=False)
patrimonio_total_evolucion.to_pickle(f'{dir_declaraciones}/evolucion_patrimonio/patrimonio_total_evolucion.pkl')
patrimonio_total_evolucion

,curp,year,valor_vehiculos,valor_inmuebles,valor_inversiones,patrimonio_total
0,ABAF800309HPURÑT08,2015,123065.0,0.0,0.0,123065.0
1,ABAF800309HPURÑT08,2016,332653.0,0.0,23111.0,355764.0
2,ABAF800309HPURÑT08,2017,479765.0,0.0,18488.0,498253.0
3,ABAF800309HPURÑT08,2018,605547.0,0.0,15529.0,621076.0
4,ABAF800309HPURÑT08,2019,695333.0,0.0,18479.0,713812.0
...,...,...,...,...,...,...
35014,ÑAUM890715HAGÑRR09,2017,1548106.0,5844712.0,733828.0,8126646.0
35015,ÑAUM890715HAGÑRR09,2018,1932839.0,5844712.0,706957.0,8484508.0
35016,ÑAUM890715HAGÑRR09,2019,2490573.0,5844712.0,741521.0,9076806.0
35017,ÑAUM890715HAGÑRR09,2020,2490573.0,5844712.0,753992.0,9089277.0


### Declaración de intereses

* Parece que viene vacía

## Funcionarios en contratación

In [4]:
with open(f'{dir_declaraciones}/SistemaS2.json', encoding='UTF-8') as jsonfile:
    file_fun_cont = json.load(jsonfile)
    df_fun_contra = pd.json_normalize(file_fun_cont)
    
df_fun_contra['institucion_merge'] = df_fun_contra['institucionDependencia.nombre'].str.lower().str.replace('\W', '').apply(remove_accents)

df_fun_contra.to_csv(f'{dir_declaraciones}/funcionarios_contrataciones_2021.csv', index=False)
df_fun_contra.to_pickle(f'{dir_declaraciones}/funcionarios_contrataciones_2021.pkl')

df_fun_contra.head()

<ipython-input-4-e6c8f078a4ac>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fun_contra['institucion_merge'] = df_fun_contra['institucionDependencia.nombre'].str.lower().str.replace('\W', '').apply(remove_accents)


,id,fechaCaptura,ejercicioFiscal,rfc,curp,nombres,primerApellido,segundoApellido,tipoArea,nivelResponsabilidad,tipoProcedimiento,observaciones,ramo.clave,ramo.valor,genero.clave,genero.valor,institucionDependencia.nombre,institucionDependencia.siglas,institucionDependencia.clave,puesto.nombre,puesto.nivel,superiorInmediato.nombres,superiorInmediato.primerApellido,superiorInmediato.segundoApellido,superiorInmediato.curp,superiorInmediato.rfc,superiorInmediato.puesto.nombre,superiorInmediato.puesto.nivel,puesto,superiorInmediato.genero.clave,superiorInmediato.genero.valor,institucion_merge
0,8a54a1ce-98b4-40f9-a52c-a272ed56f238,2018-05-21T17:32:28Z,2018,ROCA9005303Z8,ROCA900530MYNDLL04,Alejandra,Rodríguez,Calderón,"[{'clave': 'T', 'valor': 'TÉCNICA'}, {'clave':...","[{'clave': 'R', 'valor': 'RESOLUCIÓN'}]","[{'clave': 1, 'valor': ''}]",,23,Provisiones salariales y económicas,F,FEMENINO,SECRETARÍA DE ENERGÍA,SENER,XYZ987,DIRECTOR DE ADMINISTRACIÓN DE BIENES INFORMÁTICOS,M11,PEDRO,CRUZ,MEDINA,CUMP770826HMNRDD96,CUMP7708269R6,Director General,CFM2300003,NaN,NaN,NaN,secretariadeenergia
1,1d6f5b03-0e18-4b1b-8b73-e11680a89bb8,2016-06-03T07:07:26Z,2016,CÓGP760116E91,CÓGP760116HAGRVT08,Patricio,Córdova,Guevara,"[{'clave': 'RC', 'valor': 'RESPONSABLE DE LA C...","[{'clave': 'A', 'valor': 'ATENCIÓN'}]","[{'clave': 1, 'valor': 'CONTRATACIONES PÚBLICA...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIO DE INFORMACION AGROALIMENTARIA Y PESQ...,ICIO,ERV,NaN,NaN,Safiya,MONTOUTO,TALSI,MOTS831018MDFNLF54,MOTS831018A14,Director General,K31,Jefe de Departamento,F,FEMENINO,serviciodeinformacionagroalimentariaypesquera
2,13c81925-aa43-4c11-834c-7ceaca3f1ed1,2016-02-07T05:17:02Z,2016,CASC850118D51,CASC850118HVZRLR03,Carlos,Caraballo,Saldaña,[],"[{'clave': 'O', 'valor': 'OTRO'}, {'clave': 'A...","[{'clave': 4, 'valor': 'ASIGNACIÓN Y EMISIÓN D...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIOS DE ALMACENAMIENTO DEL NORTE S.A.,ICIO,ERV,NaN,NaN,Raizy,DAGUERRE,LAVIADA,DALR971211MOXGVZ21,DALR971211A46,Jefe de departamento,O21,Secretario/a de Estado A,F,FEMENINO,serviciosdealmacenamientodelnortesa
3,2f52789d-31d1-4299-9fcf-19484f4af554,2021-11-21T12:22:02Z,2021,FOOF890119E26,FOOF890119MAGNZL09,Florencia,Fonseca,Orozco,[],"[{'clave': 'A', 'valor': 'ATENCIÓN'}]",[],NaN,28,Participaciones a Entidades Federativas y Muni...,F,FEMENINO,INSTITUTO NACIONAL DE BELLAS ARTES Y LITERATURA,ITUT,NST,NaN,NaN,Haydn,FORADADA,MONTESDEOCA,FOMH750903HVZRNY66,FOMH750903C34,Subdirector,N31,Director General Paraestatal A,M,MASCULINO,institutonacionaldebellasartesyliteratura
4,c2e3ebf4-90c5-4ee0-8e4e-c462f4db04ec,2020-06-02T21:43:31Z,2020,NAKG810014A41,NAKG810014MSNVNR04,Gregorio,Navarro,Kano,"[{'clave': 'R', 'valor': 'REQUIRENTE'}, {'clav...","[{'clave': 'A', 'valor': 'ATENCIÓN'}, {'clave'...","[{'clave': 2, 'valor': 'CONCESIONES, LICENCIAS...",NaN,28,Participaciones a Entidades Federativas y Muni...,F,FEMENINO,COMISION NACIONAL DE ACUACULTURA Y PESCA,SION,OMI,NaN,NaN,Emiliana,GONDA,MAREGA,GOME890501MVZNRL70,GOME890501E15,Jefe de departamento,O21,Subdirector de Area,F,FEMENINO,comisionnacionaldeacuaculturaypesca


## Funcionarios sancionados

In [5]:
with open(f'{dir_declaraciones}/Sistema3Servidores.json', encoding='UTF-8') as jsonfile:
    file_fun_sanc = json.load(jsonfile)
    df_fun_sanc = pd.json_normalize(file_fun_sanc)
    
df_fun_sanc['institucion_merge'] = df_fun_contra['institucionDependencia.nombre'].str.lower().str.replace('\W', '').apply(remove_accents)
df_fun_sanc.to_csv(f'{dir_declaraciones}/funcionarios_sancionados_2021.csv', index=False)
df_fun_sanc.to_pickle(f'{dir_declaraciones}/funcionarios_sancionados_2021.pkl')
df_fun_sanc.head()

<ipython-input-5-89dca63c4f85>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fun_sanc['institucion_merge'] = df_fun_contra['institucionDependencia.nombre'].str.lower().str.replace('\W', '').apply(remove_accents)


,id,fechaCaptura,expediente,autoridadSancionadora,tipoSancion,causaMotivoHechos,observaciones,institucionDependencia.nombre,servidorPublicoSancionado.rfc.rfc,servidorPublicoSancionado.rfc.homoClave,servidorPublicoSancionado.curp,servidorPublicoSancionado.nombres,servidorPublicoSancionado.primerApellido,servidorPublicoSancionado.segundoApellido,servidorPublicoSancionado.genero.clave,servidorPublicoSancionado.genero.valor,servidorPublicoSancionado.puesto,tipoFalta.clave,tipoFalta.valor,tipoFalta.descripcion,resolucion.fechaResolucion,multa.moneda.clave,multa.moneda.valor,multa.monto,inhabilitacion.plazo,inhabilitacion.fechaInicial,inhabilitacion.fechaFinal,servidorPublicoSancionado.rfc,institucion_merge
0,a94cacc9-53ba-40d1-bc04-08b765010ac0,2010-01-01,SANC/bed70faf,Contraloría Interna,"[{'clave': 'SE', 'valor': 'SANCIÓN ECONÓMICA',...",DESVÍO DE RECURSOS PÚBLICOS,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,PUDD930927,E33,PUDD930927MVZG N02,Daniela,Puga,de Anda,M,FEMENINO,Jefe de departamento,ASEX,COMETER O TOLERAR CONDUCTAS DE ACOSO SEXUAL,COMETER O TOLERAR CONDUCTAS DE ACOSO SEXUAL,2010-01-01,MXN,Peso Mexicano,13185.0,NaN,NaN,NaN,NaN,secretariadeenergia
1,6902edeb-1a0a-4ccf-af66-6b9308e0e173,2010-03-01,EXP/80ca6e1b,Contraloría Interna,"[{'clave': 'IRSC', 'valor': 'INDEMNIZACIÓN RES...",ABUSO DE FUNCIONES,Lorem Ipsum es simplemente el texto de relleno...,Comisión Federal de Electricidad,VICJ950414,E62,VICJ950414MVZLRS02,Jose Daniel,Villarreal,Carrion,M,FEMENINO,Jefe de departamento,AC,ACTO DE CORRUPCIÓN,RECIBIR SOBORNO POR UN TRÁMITE,2010-01-01,MXN,Peso Mexicano,9074.0,NaN,NaN,NaN,NaN,serviciodeinformacionagroalimentariaypesquera
2,e5f8626a-fa7f-41aa-bbe7-4685473ca62f,2010-04-01,SANC/23af791f,Contraloría Interna,"[{'clave': 'I', 'valor': 'INHABILITADO', 'desc...",TRÁFICO DE INFLUENCIAS,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,GAER730906,B58,GAER730906MVZSHC05,Ricardo,Gastélum,Echevarría,M,FEMENINO,Jefe de departamento,ENCB,ENCUBRIMIENTO,ENCUBRIMIENTO,2010-01-01,NaN,NaN,NaN,6 años,2010-01-01,2016-00-01,NaN,serviciosdealmacenamientodelnortesa
3,24700b4b-9fa1-4d46-8a84-93a53a66fe82,2010-04-31,GCPD/b0b7b091,Contraloría Interna,"[{'clave': 'M', 'valor': 'MULTADO', 'descripci...","COHECHO, ACEPTO BENEFICIO NO COMPRENDIDO EN SU...","Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,GAER730906,B58,GAER730906MVZSHC05,Ricardo,Gastélum,Echevarría,M,FEMENINO,Jefe de departamento,OTRO,OTRO,OTRO,2010-01-01,MXN,Peso Mexicano,21956.0,NaN,NaN,NaN,NaN,institutonacionaldebellasartesyliteratura
4,a89fa644-deea-49a0-883c-314cd166ceff,2010-01-01,GCPD/01a89518,Tribunal,"[{'clave': 'O', 'valor': 'OTRO', 'descripcion'...",CONTRATACIÓN INDEBIDA,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,RAQR910021,D72,RAQR910021MAGMRM06,Ramón,Ramos,Quiroz,M,FEMENINO,Jefe de departamento,IDSP,INCUMPLIMIENTO EN DECLARACION DE SITUACION PAT...,OCULTAMIENTODE PROPIEDADES,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,comisionnacionaldeacuaculturaypesca


## Riesgo de contratación IMCO

### Puntaje

In [8]:
imco = pd.read_excel('datos/imco/IRC 2018-2020_Instituciones.xlsx', sheet_name='Análisis 2018-2020')
imco['institucion_merge'] = imco['INSTITUCION'].str.lower().str.replace('\W', '').apply(remove_accents)
imco.to_csv(f'{dir_declaraciones}/IRC_IMCO_2021.csv', index=False)
imco.to_pickle(f'{dir_declaraciones}/IRC_IMCO_2021.pkl')
imco.head()

<ipython-input-8-811145e2ccb3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  imco['institucion_merge'] = imco['INSTITUCION'].str.lower().str.replace('\W', '').apply(remove_accents)


,INSTITUCION,SIGLAS,Sector,IRC-2020,IRC-2019,IRC-2018,Progreso 19-20,Progreso 18-20,Monto-2020,Rango 2020,Monto-2019,Rango-2019,Monto-2018,Rango-2018,Diferencia Monto 18-20,Diferencia % 18-20,Diferencia Monto 19-20,Diferencia % 19-20,Riesgo Anomalías 2020,Riesgo Anomalías 2019,Riesgo Anomalías 2018,Riesgo Competencia 2020,Riesgo Competencia 2019,Riesgo Competencia 2019.1,Riesgo Transparencia 2020,Riesgo Transparencia 2019,Riesgo Transparencia 2018,institucion_merge
0,Servicio de Administración y Enajenación de Bi...,SAE,HACIENDA Y CRÉDITO PÚBLICO,NaN,NaN,18.900422,NaN,NaN,NaN,NaN,NaN,NaN,2.518243e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,NaN,20.495994,NaN,NaN,30.443397,NaN,NaN,19.779828,serviciodeadministracionyenajenaciondebienes
1,"Consejo de Promoción Turística de México, S.A....",CPTM,TURISMO,NaN,45.36129,31.813780,NaN,NaN,NaN,NaN,6590676.52,Monto bajo,1.984737e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,38.959828,27.691786,NaN,73.401238,56.095421,NaN,20.204563,23.530833,consejodepromocionturisticademexicosadecv
2,Policía Federal,PF,seguridad,NaN,NaN,40.859670,NaN,NaN,NaN,NaN,NaN,NaN,1.266091e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,NaN,55.000296,NaN,NaN,55.031264,NaN,NaN,27.141366,policiafederal
3,Procuraduría General de la República,PGR,PGR,NaN,NaN,33.909026,NaN,NaN,NaN,NaN,NaN,NaN,9.511073e+08,Monto medio,NaN,NaN,NaN,NaN,NaN,NaN,26.345552,NaN,NaN,32.596300,NaN,NaN,43.187765,procuraduriageneraldelarepublica
4,Banco del Ahorro Nacional y Servicios Financie...,BANSEFI,HACIENDA Y CRÉDITO PÚBLICO,NaN,NaN,22.231333,NaN,NaN,NaN,NaN,NaN,NaN,7.508318e+08,Monto medio,NaN,NaN,NaN,NaN,NaN,NaN,33.933295,NaN,NaN,48.324705,NaN,NaN,7.656397,bancodelahorronacionalyserviciosfinancierossnc


### Indicadores por año

In [31]:
imco_por_year = pd.concat([pd.read_excel('datos/imco/IRC 2018-2020_Instituciones.xlsx', sheet_name=f'INST{y}') for y in range(2018, 2021)])\
    .rename(columns=lambda x: remove_accents(x.lower().replace(' ', '_').replace('-', '_')))
imco_por_year.to_csv(f'{dir_declaraciones}/irc_imco_x_year.csv', index=False)
imco_por_year.to_pickle(f'{dir_declaraciones}/irc_imco_x_year.pkl')
imco_por_year.head()

,contador,institucion,siglas,ano,irc_institucion,monto_institucion,porcentaje_del_monto_institucion,riesgo_inst_anomalias,riesgo_inst_competencia,riesgo_inst_transparencia,clasificacion_del_gasto
0,7,Instituto Mexicano del Seguro Social,IMSS,2018,25.659125,9.433604e+10,0.241052,36.162275,30.396577,24.390020,Monto muy alto
1,10,Secretaría de Comunicaciones y Transportes,SCT,2018,28.126668,5.682986e+10,0.145214,28.287899,26.279109,35.576533,Monto muy alto
2,3,Instituto de Seguridad y Servicios Sociales de...,ISSSTE,2018,42.773557,2.947029e+10,0.075304,45.829984,33.992884,48.834365,Monto muy alto
3,177,"Grupo Aeroportuario de la Ciudad de México, S....",GACM,2018,25.923394,2.519830e+10,0.064388,11.076675,46.619548,27.125676,Monto muy alto
4,114,Servicio de Administración Tributaria,SAT,2018,15.284201,1.846569e+10,0.047184,13.737414,32.755914,15.022689,Monto muy alto


In [30]:
imco_por_year['ano'].value_counts()

2019    268
2020    263
2018    262
Name: ano, dtype: int64

### Por contrataciones
* Número de contratos
* Monto total de contratación
* Contratos por invitación restringida

In [41]:
#imco_x_contrato = pd.read_excel('datos/imco/Indicadores_contratos_IRC.xlsx', sheet_name='Contratos.con.variables')
imco_x_contrato['contratacion_restringida'] = imco_x_contrato['Tipo.de.procedimiento'].isin(['ADJUDICACIÓN DIRECTA', 'INVITACIÓN A CUANDO MENOS 3 PERSONAS'])
imco_x_contrato.head()

,Siglas.de.la.Institución,Institución,Nombre.de.la.UC,Clave.de.la.UC,Tipo.de.contratación,Tipo.de.procedimiento,Fundamento.legal,Sin.justificación,Rebasa.monto,Número.del.procedimiento,Carácter.del.procedimiento,Importe.pesos,Proveedor.o.contratista,RFC,Folio.en.el.RUPC,Estratificación.de.la.empresa,Diferencia.creación.contrato,Reciente.creación,RFC.69bis,Proveedor_Sancionado,Código.del.contrato,Título.del.contrato,Descripción.del.contrato,CUCOP,Objeto,Fecha.de.inicio.del.contrato,Fecha.de.fin.del.contrato,Contrato.marco,Compra.consolidada,Forma.de.participación,Participantes,Ausencia.Competencia,testigo_social,exclusivo_mipymes,exclusivo_mipymes_no_respetado,Fecha.de.publicación,Fecha.de.apertura,Fecha.de.fallo,Plazos.Cortos,Publicación.Tardía,Dirección.del.anuncio,Link.Funcional,Publicación.EDCA,Información_Scrapper,archivo_convocatoria,archivo_junta,archivo_apertura,archivo_fallo,archivo_contrato,numero_archivos,numero_convenios,AD.sin.contrato,IR.sin.documento,LP.sin.documento,IHH_Institución,Calificación IRC,Calificación Competencia,Calificación Transparencia,Calificación Anomalías
0,HG,"Hospital General ""Dr. Manuel Gea González""",HG-Subdirección de Recursos Materiales #012NBB002,012NBB002,Adquisiciones,ADJUDICACIÓN DIRECTA,Art. 42 párrafo primero,0.0,0.0,AA-012NBB002-E52-2020,Nacional,146250.0,LA BODEGUITA DE LOS SUEÑOS SA DE CV,BSU140306EW8,415080.0,Pequeña,2304.0,0.0,0,0,2376723.0,PAPELERIA 2020,papeleria 2020,2110.0,"Materiales, utiles y equipos menores de oficina",2020-06-26,2020-12-31,No se utilizó el Contrato Marco,0,Mixta,NaN,NaN,0,0,NaN,2020-06-12,2020-06-16 00:00:00,NaN,NaN,0.0,https://compranet.hacienda.gob.mx/esop/guest/g...,1,1,1,0,0,0,0,0,0,0,1.0,NaN,NaN,1176.837507,26.049074,34.369677,12.679517,30.453719
1,CENACE,Centro Nacional de Control de Energía,CENACE-Gerencia de Control Regional Noreste #0...,018TOM997,Servicios,ADJUDICACIÓN DIRECTA,Art. 42 párrafo primero,0.0,0.0,AA-018TOM997-E7-2020,Nacional,51062.0,POWER SYSTEMS SERVICE S.A. DE C.V.,PSS990127RE5,4189.0,Pequeña,7908.0,0.0,0,0,2376740.0,MANTENIMIENTO A CLIMAS DE PRECISIÓN DE LA GCRNE,MANTENIMIENTO A CLIMAS DE PRECISIÓN DE LA GCRNE,3570.0,"Instalacion, reparacion y mantenimiento de maq...",2020-09-21,2020-09-25,No se utilizó el Contrato Marco,0,Electrónica,NaN,NaN,0,0,NaN,2020-08-06,2020-08-25 00:00:00,NaN,NaN,0.0,https://compranet.hacienda.gob.mx/esop/guest/g...,1,1,1,0,0,0,0,0,8,0,1.0,NaN,NaN,4810.490558,25.779687,40.531878,13.846593,14.276276
2,INER,Instituto Nacional de Enfermedades Respiratori...,INER-Departamento de Adquisiciones #012NCD001,012NCD001,Adquisiciones,ADJUDICACIÓN DIRECTA,Art. 42 párrafo primero,0.0,0.0,AA-012NCD001-E507-2020,Nacional,91822.5,KENDALL DE MEXICO S.A DE C.V.,KME590502HY7,6183.0,No MIPYME,-14106.0,0.0,0,0,2376835.0,DEPARTAMENTO DE INGENIERÍA BIOMÉDICA,DEPARTAMENTO DE INGENIERÍA BIOMÉDICA\n\n\n\n,2950.0,Refacciones y accesorios menores de equipo e i...,2020-09-17,2020-12-31,No se utilizó el Contrato Marco,0,Presencial,NaN,NaN,0,0,NaN,2020-09-24,2020-09-17 00:00:00,NaN,NaN,1.0,https://compranet.hacienda.gob.mx/esop/guest/g...,1,1,1,0,0,0,0,1,0,0,0.0,NaN,NaN,476.419078,32.390872,46.910429,11.572610,35.160766
3,DICONSA,"Diconsa, S.A. de C.V.",DICONSA-Dirección Comercial #008VSS998,008VSS998,Adquisiciones,ADJUDICACIÓN DIRECTA,Art. 41 fr. XII,0.0,NaN,AA-008VSS998-E12801-2020,Nacional,127440.0,"INDUSTRIAL PATRONA, S.A. DE C.V.",NaN,NaN,NaN,NaN,NaN,0,0,2376228.0,ACEITE VEGETAL COMESTIBLE,Compra para atender Requerimiento de Abasto Ru...,2380.0,Mercancias adquiridas para su comercializacion,2020-08-21,2020-08-31,No se utilizó el Contrato Marco,0,Presencial,NaN,NaN,0,0,NaN,2020-09-23,2020-08-21 00:00:00,NaN,NaN,1.0,https://compranet.hacienda.gob.mx/esop/guest/g...,1,1,1,0,0,0,0,1,0,0,0.0,NaN,NaN,148.551658,36.558332,48.726338,11.532144,47.407544
4,DICONSA,"Diconsa, S.A. de C.V.",DICONSA-Dirección Comercial #008VSS998,008VSS998,Adquisiciones,ADJUDICACIÓN DIRECTA,Art. 41 fr. XII,0.0,NaN,AA-008VSS998-E12889-2020,Nacional,288732.7

In [68]:
datos_imco = imco_x_contrato.groupby(['Siglas.de.la.Institución', 'Institución'], as_index=False).agg(
    n_contratos=('Código.del.contrato', 'count'),
    total_monto=('Importe.pesos', 'sum'),
    contratacion_restringida=('contratacion_restringida', 'mean'),
    proveedor_sancionado=('Proveedor_Sancionado', 'mean'),
    ausencia_competencia=('Ausencia.Competencia', 'mean'),
    plazos_cortos=('Plazos.Cortos', 'mean'),
    publicacion_tardia=('Publicación.Tardía', 'mean'),
    ihh_institucion=('IHH_Institución', 'max'),
    calificacion_irc=('Calificación IRC', 'max'),
    calificacion_competencia=('Calificación Competencia', 'max'),
    calificacion_transparencia=('Calificación Transparencia', 'max'),
    calificacion_anomalias=('Calificación Anomalías', 'max')   
).assign(
    institucion_merge=lambda x: x['Institución'].str.lower().str.replace('\W', '').apply(remove_accents)
)
vars_porcentaje = ['contratacion_restringida', 'proveedor_sancionado', 'ausencia_competencia', 'plazos_cortos', 'publicacion_tardia']
datos_imco[vars_porcentaje] = datos_imco[vars_porcentaje].mul(100).round(2)
datos_imco.head()
datos_imco.to_csv(f'{dir_declaraciones}/datos_imco.csv', index=False)
datos_imco.to_pickle(f'{dir_declaraciones}/datos_imco.pkl')
datos_imco.head()

<ipython-input-68-7f1d7d37e4d8>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  institucion_merge=lambda x: x['Institución'].str.lower().str.replace('\W', '').apply(remove_accents)


,Siglas.de.la.Institución,Institución,n_contratos,total_monto,contratacion_restringida,proveedor_sancionado,ausencia_competencia,plazos_cortos,publicacion_tardia,ihh_institucion,calificacion_irc,calificacion_competencia,calificacion_transparencia,calificacion_anomalias,institucion_merge
0,AEFCM,Autoridad Educativa Federal en la Ciudad de Mé...,364,3.603601e+09,84.89,0.00,3904.76,33.10,48.48,3338.410682,44.316575,80.565191,24.549848,28.418383,autoridadeducativafederalenlaciudaddemexico
1,AEM,Agencia Espacial Mexicana,53,1.881035e+07,92.45,0.00,NaN,50.00,86.79,3694.116518,83.138402,65.713309,76.575925,58.997048,agenciaespacialmexicana
2,AGN,Archivo General de la Nación,175,2.085810e+08,78.86,0.00,3745.61,12.00,62.29,2458.469655,20.847346,49.005965,8.477071,32.127685,archivogeneraldelanacion
3,AGROASEMEX,"Agroasemex, S.A.",215,9.357466e+07,86.05,0.00,3304.12,0.00,75.35,496.257558,24.353208,48.435434,18.914703,40.100469,agroasemexsa
4,AICM,Aeropuerto Internacional de la Ciudad de Méxic...,617,8.713214e+09,59.32,0.49,2952.95,29.43,30.96,909.989742,17.874138,43.081730,11.238823,36.341437,aeropuertointernacionaldelaciudaddemexicosadecv


# Análisis de datos

## Informacion general de funcionarios

In [149]:
df_funcionarios = pd.read_pickle(f'{dir_declaraciones}/datos_funcionarios_2021.pkl')
df_funcionarios.head()

,curp,nombre,metadata.institucion,nivelOrdenGobierno,declaracion.situacionPatrimonial.datosEmpleoCargoComision.areaAdscripcion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.contratadoPorHonorarios,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision,edu_nivel,edu_doc,edu_esp,edu_lic,edu_mae,exp_privado,exp_publico,exp_total,n_vehiculos,n_vehiculos_lujo,valor_vehiculos,n_bienes,valor_bienes,valor_inversiones,patrimonio_total,ingreso_total_neto_2015,ingreso_total_neto_2016,ingreso_total_neto_2017,ingreso_total_neto_2018,ingreso_total_neto_2019,ingreso_total_neto_2020
0,ABAF800309HPURÑT08,Fatima Abreu Acuña,SERVICIO DE PROTECCIÓN FEDERAL,FEDERAL,Dirección General de Administración,False,O32,LICENCIATURA,0,0,1,0,12.1,0.0,12.1,6.0,2.0,695333.0,0.0,0.0,10954.0,0.0,105560.0,506688.0,506688.0,506688.0,506688.0,506688.0
1,ABAG780123HVZYRL05,Gilberto Abeyta Adorno,COMISION NACIONAL DEL SISTEMA DE AHORRO PARA E...,ESTATAL,Unidad de Asuntos Jurídicos,False,18,DOCTORADO,1,1,1,0,12.3,0.0,12.3,5.0,2.0,4359275.0,2.0,170266978.0,59315459.0,233941712.0,1514535.0,4854048.0,6030672.0,5115900.0,5262936.0,4329060.0
2,ABAM750311HPURCR03,Marcela Abreu Alcántar,ADMINISTRACION PORTUARIA INTEGRAL DE DOS BOCAS...,MUNICIPAL,Dirección General de Vinculación,False,A33,LICENCIATURA,0,0,1,0,6.2,3.9,10.1,7.0,0.0,1646379.0,0.0,0.0,10186.0,0.0,125000.0,600000.0,600000.0,600000.0,600000.0,600000.0
3,ABAM881010MDFYVT07,Matías Abeyta Alvarado,"FONATUR OPERADORA PORTUARIA, S.A. DE C.V.",ESTATAL,Unidad de Transparencia,False,M34,LICENCIATURA,0,0,1,0,10.4,0.0,10.4,4.0,1.0,1756205.0,1.0,2605031.0,827536.0,5188772.0,357725.0,1624440.0,1595808.0,1698564.0,1631928.0,1588536.0
4,ABBA870314MSNRRN07,Adán Abrego Bernal,TRANSPORTADORA DE SAL S.A. DE C.V.,MUNICIPAL,Unidad de Asuntos Jurídicos,False,N34,MAESTRÍA,0,0,1,1,10.2,0.0,10.2,6.0,0.0,2657856.0,1.0,4937025.0,822358.0,8417239.0,315750.0,1314072.0,1106616.0,1302660.0,1179192.0,1177404.0


## Declaraciones

In [17]:
decl = pd.read_pickle(f'{dir_declaraciones}/declaraciones_2021.pkl')\
    .sort_values(['curp', 'metadata.actualizacion'])
decl.head()

,id,_id.$oid,metadata.actualizacion,metadata.tipo,metadata.institucion,metadata.declaracionCompleta,metadata.actualizacionConflictoInteres,declaracion.situacionPatrimonial.datosGenerales.nombre,declaracion.situacionPatrimonial.datosGenerales.primerApellido,declaracion.situacionPatrimonial.datosGenerales.segundoApellido,curp,declaracion.situacionPatrimonial.datosGenerales.rfc.rfc,declaracion.situacionPatrimonial.datosGenerales.rfc.homoClave,declaracion.situacionPatrimonial.datosGenerales.correoElectronico.institucional,declaracion.situacionPatrimonial.datosGenerales.correoElectronico.personal,declaracion.situacionPatrimonial.datosGenerales.telefono.casa,declaracion.situacionPatrimonial.datosGenerales.telefono.celularPersonal,declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.clave,declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.valor,declaracion.situacionPatrimonial.datosGenerales.paisNacimiento,declaracion.situacionPatrimonial.datosGenerales.nacionalidad,declaracion.situacionPatrimonial.datosGenerales.aclaracionesObservaciones,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.calle,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.numeroExterior,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.numeroInterior,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.coloniaLocalidad,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.municipioAlcaldia.clave,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.municipioAlcaldia.valor,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.entidadFederativa.clave,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.entidadFederativa.valor,declaracion.situacionPatrimonial.domicilioDeclarante.domicilioMexico.codigoPostal,declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad,declaracion.situacionPatrimonial.datosCurricularesDeclarante.aclaracionesObservaciones,declaracion.situacionPatrimonial.datosEmpleoCargoComision.tipoOperacion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelOrdenGobierno,declaracion.situacionPatrimonial.datosEmpleoCargoComision.ambitoPublico,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico,declaracion.situacionPatrimonial.datosEmpleoCargoComision.areaAdscripcion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.empleoCargoComision,declaracion.situacionPatrimonial.datosEmpleoCargoComision.contratadoPorHonorarios,declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision,declaracion.situacionPatrimonial.datosEmpleoCargoComision.funcionPrincipal,declaracion.situacionPatrimonial.datosEmpleoCargoComision.fechaTomaPosesion,declaracion.situacionPatrimonial.datosEmpleoCargoComision.telefonoOficina.telefono,declaracion.situacionPatrimonial.datosEmpleoCargoComision.telefonoOficina.extension,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.calle,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.numeroExterior,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.numeroInterior,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.coloniaLocalidad,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.municipioAlcaldia.clave,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.municipioAlcaldia.valor,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.entidadFederativa.clave,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.entidadFederativa.valor,declaracion.situacionPatrimonial.datosEmpleoCargoComision.domicilioMexico.codigoPostal,declaracion.situacionPatrimonial.datosEmpleoCargoComision.aclaracionesObservaciones,declaracion.situacionPatrimonial.experienciaLaboral.ninguno,declaracion.situacionPatrimonial.experienciaLaboral.exp

In [25]:
decl.notnull().sum().to_dict()

{'id': 40019,
 '_id.$oid': 40019,
 'metadata.actualizacion': 40019,
 'metadata.tipo': 40019,
 'metadata.institucion': 40019,
 'metadata.declaracionCompleta': 40019,
 'metadata.actualizacionConflictoInteres': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.nombre': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.primerApellido': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.segundoApellido': 40019,
 'curp': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.rfc.rfc': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.rfc.homoClave': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.correoElectronico.institucional': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.correoElectronico.personal': 40019,
 'declaracion.situacionPatrimonial.datosGenerales.telefono.casa': 40016,
 'declaracion.situacionPatrimonial.datosGenerales.telefono.celularPersonal': 40016,
 'declaracion.situacionPatrimonial.datosGenerales.situacionPersonalEstadoCivil.c

In [4]:
decl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40019 entries, 0 to 40018
Columns: 258 entries, id to institucion_merge
dtypes: bool(4), float64(41), object(213)
memory usage: 77.7+ MB


In [10]:
decl['declaracion.situacionPatrimonial.datosGenerales.curp'].duplicated().sum()

35014

### Educacion

In [51]:
educacion = pd.read_pickle(f'{dir_declaraciones}/educacion_declaraciones_2021.pkl')
educacion.head()

,tipoOperacion,carreraAreaConocimiento,status,documentoObtenido,fechaObtencion,nivel.clave,nivel.valor,institucionEducativa.nombre,institucionEducativa.ubicacion,curp,estatus
0,SIN_CAMBIOS,Licenciatura en Comercio Internacional,FINALIZADO,TITULO,2008-08-15T07:17:26.107Z,LIC,LICENCIATURA,Universidad Tecnológica de México,MX,ABAF800309HPURÑT08,NaN
0,SIN_CAMBIOS,Licenciatura en Economía,FINALIZADO,TITULO,2001-01-27T03:53:39.906Z,LIC,LICENCIATURA,Universidad Panamericana,MX,ABAG780123HVZYRL05,NaN
1,SIN_CAMBIOS,Arte y Diseño,FINALIZADO,CONSTANCIA,2007-10-29T04:35:15.410Z,ESP,ESPECIALIDAD,Massachusetts Institute of Technology,EX,ABAG780123HVZYRL05,NaN
2,SIN_CAMBIOS,"Ingeniería, Arquitectura, Urbanismo y afines",FINALIZADO,CONSTANCIA,2013-09-08T12:10:24.126Z,DOC,DOCTORADO,Universidad Panamericana,MX,ABAG780123HVZYRL05,NaN
0,SIN_CAMBIOS,Licenciatura en Administración de las Tecnolog...,FINALIZADO,TITULO,2011-05-14T09:19:56.468Z,LIC,LICENCIATURA,Universidad Justo Sierra,MX,ABAM750311HPURCR03,NaN


### Experiencia

In [5]:
experiencia = pd.read_pickle(f'{dir_declaraciones}/experiencia_declaraciones_2021.pkl')
experiencia.head()

,tipoOperacion,nombreEmpresaSociedadAsociacion,rfc,area,puesto,fechaIngreso,fechaEgreso,ubicacion,ambitoSector.clave,ambitoSector.valor,sector.clave,sector.valor,curp,nivelOrdenGobierno,ambitoPublico,nombreEntePublico,areaAdscripcion,empleoCargoComision,funcionPrincipal,experiencia
0,SIN_CAMBIOS,Grupo Financiero Scotiabank Inverlat,GRUP881113AB2,Comercial,Atención a clientes,2000-12-20T07:12:55.598Z,2006-10-30T03:42:00.869Z,MX,PRV,PRIVADO,OTRO,Servicios financieros,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,2139
1,SIN_CAMBIOS,FEMSA,FEMS781119CC3,Comercial,Agente de ventas,2007-08-05T00:36:01.639Z,2010-04-17T09:25:49.967Z,MX,PRV,PRIVADO,OTRO,Holding,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,986
2,SIN_CAMBIOS,Grupo BAL,GRUP751016CX2,Recursos Humanos,Repartidor,2011-03-02T04:19:03.989Z,2014-09-26T23:58:27.462Z,MX,PRV,PRIVADO,OTRO,Holding,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,1304
0,SIN_CAMBIOS,Grupo Televisa,GRUP881111AC1,Gerencia,Gerente Regional,2001-01-29T14:04:42.439Z,2006-11-12T13:42:57.818Z,MX,PRV,PRIVADO,OTRO,Telecomunicaciones,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,2112
1,SIN_CAMBIOS,Grupo Financiero BBVA Bancomer,GRUP861126AC1,Comercial,Gerente,2007-09-19T13:10:57.858Z,2010-11-06T05:31:45.396Z,MX,PRV,PRIVADO,OTRO,Servicios financieros,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,1143


### Dependientes

In [6]:
dependientes = pd.read_pickle(f'{dir_declaraciones}/dependientes_declaraciones_2021.pkl')
dependientes.head()

,tipoOperacion,nombre,primerApellido,segundoApellido,fechaNacimiento,extranjero,curp,habitaDomicilioDeclarante,lugarDondeReside,rfc.rfc,rfc.homoClave,parentescoRelacion.clave,parentescoRelacion.valor,actividadLaboral.clave,actividadLaboral.valor,curp_funcionario,rfc,domicilioMexico.calle,domicilioMexico.numeroExterior,domicilioMexico.numeroInterior,domicilioMexico.coloniaLocalidad,domicilioMexico.municipioAlcaldia.clave,domicilioMexico.municipioAlcaldia.valor,domicilioMexico.entidadFederativa.clave,domicilioMexico.entidadFederativa.valor,domicilioMexico.codigoPostal,domicilioExtranjero.calle,domicilioExtranjero.numeroExterior,domicilioExtranjero.numeroInterior,domicilioExtranjero.ciudadLocalidad,domicilioExtranjero.estadoProvincia,domicilioExtranjero.pais,domicilioExtranjero.codigoPostal,actividadLaboralSectorPublico.nivelOrdenGobierno,actividadLaboralSectorPublico.ambitoPublico,actividadLaboralSectorPublico.nombreEntePublico,actividadLaboralSectorPublico.areaAdscripcion,actividadLaboralSectorPublico.empleoCargoComision,actividadLaboralSectorPublico.funcionPrincipal,actividadLaboralSectorPublico.salarioMensualNeto.valor,actividadLaboralSectorPublico.salarioMensualNeto.moneda,actividadLaboralSectorPublico.fechaIngreso,actividadLaboralSectorPrivadoOtro.nombreEmpresaSociedadAsociacion,actividadLaboralSectorPrivadoOtro.rfc,actividadLaboralSectorPrivadoOtro.empleoCargo,actividadLaboralSectorPrivadoOtro.fechaIngreso,actividadLaboralSectorPrivadoOtro.salarioMensualNeto.valor,actividadLaboralSectorPrivadoOtro.salarioMensualNeto.moneda,actividadLaboralSectorPrivadoOtro.proveedorContratistaGobierno,actividadLaboralSectorPrivadoOtro.sector.clave,actividadLaboralSectorPrivadoOtro.sector.valor
0,SIN_CAMBIOS,Emilio,Menchaca,Guerra,2012-08-07,False,MEGE20120807MPUNRL07,True,MEXICO,MEGE20120807,A28,HIJ,HIJO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SIN_CAMBIOS,Jerónimo,Carbajal,Xacon,2011-03-12,False,CAXJ20110312MAGRCR02,True,MEXICO,CAXJ20110312,B60,PRI,PRIMO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SIN_CAMBIOS,María Eugenia,Kadar rodriguez,Irizarry,2011-07-06,False,KAIM20110706HPUDZR02,True,MEXICO,KAIM20110706,A44,PRI,PRIMO(A),NIN,NINGUNO,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SIN_CAMBIOS,Elena,Arias,Razo,2009-00-12,False,ARRE20090012MSNSZN04,True,MEXICO,ARRE20090012,D93,NIN,NINGUNO,NIN,NINGUNO,ABAG780123HVZYRL05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,SIN_CAMBIOS,Dulce María,Ornelas,Morales,2006-07-23,False,ORMD20060723HVZNRL07,True,MEXICO,ORMD20060723,C77,AHI,AHIJADO(A),NIN,NINGUNO,ABAM750311HPURCR03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Bienes

In [7]:
inversiones = pd.read_pickle(f'{dir_declaraciones}/inversiones_declaraciones_2021.pkl')
inversiones.head()

,tipoOperacion,titular,numeroCuentaContrato,porcentajeIncrementoDecremento,tipoInversion.clave,tipoInversion.valor,subTipoInversion.clave,subTipoInversion.valor,localizacionInversion.pais,localizacionInversion.institucionRazonSocial,localizacionInversion.rfc,saldoFechaConclusion.valor,saldoFechaConclusion.moneda,curp_funcionario,subTipoInversion.tipoInversion
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",87349554,-22%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Valdez, Méndez and Sanabria",68b4640a-5458-47aa-9b8c-e4ea744af3d6,10954.0,MXN,ABAF800309HPURÑT08,NaN
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",21866734,-10%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Padrón, Quintero and Fernández",f444de0b-3b7d-45b4-a6f0-9edcb782f89b,26637502.0,MXN,ABAG780123HVZYRL05,NaN
1,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",32190625,5%,BANC,BANCARIA,CTES,CERTIFICADOS DE LA TESORERÍA,MX,Barajas e Hijos,1de64040-77c8-4e66-93df-495666c4e15d,26958683.0,MXN,ABAG780123HVZYRL05,AFOT
2,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",29245153,-12%,FINV,FONDOS DE INVERSIÓN,CAHO,CUENTA DE AHORRO,MX,Melgar - Zambrana,92c6b093-b455-4c04-8282-c8867e6cb929,5719274.0,MXN,ABAG780123HVZYRL05,BANC
0,MODIFICAR,"[{'clave': 'DEC', 'valor': 'DECLARANTE'}]",22013834,-8%,BANC,BANCARIA,CNOM,CUENTA DE NÓMINA,MX,"Pulido, Gallardo and Alaniz",2c22f844-6861-48f9-91b9-189099419bd7,10186.0,MXN,ABAM750311HPURCR03,NaN


### Vehículos

In [8]:
vehiculos = pd.read_pickle(f'{dir_declaraciones}/vehiculos_declaraciones_2021.pkl')
vehiculos.head()

,tipoOperacion,marca,modelo,anio,numeroSerieRegistro,tipoVehiculo.clave,tipoVehiculo.valor,titularBien.clave,titularBien.valor,valorAdquisicion.valor,valorAdquisicion.moneda,curp_funcionario,titular,transmisor,tercero,formaPago,fechaAdquisicion,lugarRegistro.pais,lugarRegistro.entidadFederativa.clave,lugarRegistro.entidadFederativa.valor,formaAdquisicion.clave,formaAdquisicion.valor,motivoBaja.clave,motivoBaja.valor
0,BAJA,opel,manta,1974,75248817-8f2e-4168-bbf6-3bf349f34636,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,123065.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAJA,volvo,264gl,1978,e24fba0f-1c38-4955-9bef-e0d4ada7a9db,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,169142.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAJA,chevrolet,citation,1982,822fa386-d514-4f12-a211-ed47bc9b747b,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,40446.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAJA,datsun,pl510,1971,555e9710-49d5-4e67-a665-07659f62db28,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,147112.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BAJA,ford,granada ghia,1976,cca40092-3cca-44ff-9fbe-5651277434c6,AUMOT,AUTOMÓVIL/MOTOCICLETA,DEC,DECLARANTE,125782.0,MXN,ABAF800309HPURÑT08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Funcionarios en contratación

In [148]:
df_fun_contra = pd.read_pickle(f'{dir_declaraciones}/funcionarios_contrataciones_2021.pkl')\
    .rename(columns={'superiorInmediato.curp': 'curp_jefe'})

df_fun_contra.head()

,id,fechaCaptura,ejercicioFiscal,rfc,curp,nombres,primerApellido,segundoApellido,tipoArea,nivelResponsabilidad,tipoProcedimiento,observaciones,ramo.clave,ramo.valor,genero.clave,genero.valor,institucionDependencia.nombre,institucionDependencia.siglas,institucionDependencia.clave,puesto.nombre,puesto.nivel,superiorInmediato.nombres,superiorInmediato.primerApellido,superiorInmediato.segundoApellido,curp_jefe,superiorInmediato.rfc,superiorInmediato.puesto.nombre,superiorInmediato.puesto.nivel,puesto,superiorInmediato.genero.clave,superiorInmediato.genero.valor,institucion_merge
0,8a54a1ce-98b4-40f9-a52c-a272ed56f238,2018-05-21T17:32:28Z,2018,ROCA9005303Z8,ROCA900530MYNDLL04,Alejandra,Rodríguez,Calderón,"[{'clave': 'T', 'valor': 'TÉCNICA'}, {'clave':...","[{'clave': 'R', 'valor': 'RESOLUCIÓN'}]","[{'clave': 1, 'valor': ''}]",,23,Provisiones salariales y económicas,F,FEMENINO,SECRETARÍA DE ENERGÍA,SENER,XYZ987,DIRECTOR DE ADMINISTRACIÓN DE BIENES INFORMÁTICOS,M11,PEDRO,CRUZ,MEDINA,CUMP770826HMNRDD96,CUMP7708269R6,Director General,CFM2300003,NaN,NaN,NaN,secretariadeenergia
1,1d6f5b03-0e18-4b1b-8b73-e11680a89bb8,2016-06-03T07:07:26Z,2016,CÓGP760116E91,CÓGP760116HAGRVT08,Patricio,Córdova,Guevara,"[{'clave': 'RC', 'valor': 'RESPONSABLE DE LA C...","[{'clave': 'A', 'valor': 'ATENCIÓN'}]","[{'clave': 1, 'valor': 'CONTRATACIONES PÚBLICA...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIO DE INFORMACION AGROALIMENTARIA Y PESQ...,ICIO,ERV,NaN,NaN,Safiya,MONTOUTO,TALSI,MOTS831018MDFNLF54,MOTS831018A14,Director General,K31,Jefe de Departamento,F,FEMENINO,serviciodeinformacionagroalimentariaypesquera
2,13c81925-aa43-4c11-834c-7ceaca3f1ed1,2016-02-07T05:17:02Z,2016,CASC850118D51,CASC850118HVZRLR03,Carlos,Caraballo,Saldaña,[],"[{'clave': 'O', 'valor': 'OTRO'}, {'clave': 'A...","[{'clave': 4, 'valor': 'ASIGNACIÓN Y EMISIÓN D...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIOS DE ALMACENAMIENTO DEL NORTE S.A.,ICIO,ERV,NaN,NaN,Raizy,DAGUERRE,LAVIADA,DALR971211MOXGVZ21,DALR971211A46,Jefe de departamento,O21,Secretario/a de Estado A,F,FEMENINO,serviciosdealmacenamientodelnortesa
3,2f52789d-31d1-4299-9fcf-19484f4af554,2021-11-21T12:22:02Z,2021,FOOF890119E26,FOOF890119MAGNZL09,Florencia,Fonseca,Orozco,[],"[{'clave': 'A', 'valor': 'ATENCIÓN'}]",[],NaN,28,Participaciones a Entidades Federativas y Muni...,F,FEMENINO,INSTITUTO NACIONAL DE BELLAS ARTES Y LITERATURA,ITUT,NST,NaN,NaN,Haydn,FORADADA,MONTESDEOCA,FOMH750903HVZRNY66,FOMH750903C34,Subdirector,N31,Director General Paraestatal A,M,MASCULINO,institutonacionaldebellasartesyliteratura
4,c2e3ebf4-90c5-4ee0-8e4e-c462f4db04ec,2020-06-02T21:43:31Z,2020,NAKG810014A41,NAKG810014MSNVNR04,Gregorio,Navarro,Kano,"[{'clave': 'R', 'valor': 'REQUIRENTE'}, {'clav...","[{'clave': 'A', 'valor': 'ATENCIÓN'}, {'clave'...","[{'clave': 2, 'valor': 'CONCESIONES, LICENCIAS...",NaN,28,Participaciones a Entidades Federativas y Muni...,F,FEMENINO,COMISION NACIONAL DE ACUACULTURA Y PESCA,SION,OMI,NaN,NaN,Emiliana,GONDA,MAREGA,GOME890501MVZNRL70,GOME890501E15,Jefe de departamento,O21,Subdirector de Area,F,FEMENINO,comisionnacionaldeacuaculturaypesca


In [17]:
df_fun_contra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4927 entries, 0 to 4926
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 4927 non-null   object
 1   fechaCaptura                       4927 non-null   object
 2   ejercicioFiscal                    4927 non-null   object
 3   rfc                                4927 non-null   object
 4   curp                               4927 non-null   object
 5   nombres                            4927 non-null   object
 6   primerApellido                     4927 non-null   object
 7   segundoApellido                    4927 non-null   object
 8   tipoArea                           4927 non-null   object
 9   nivelResponsabilidad               4927 non-null   object
 10  tipoProcedimiento                  4927 non-null   object
 11  observaciones                      1 non-null      object
 12  ramo.c

In [18]:
df_fun_contra.notnull().sum().to_dict()

{'id': 4927,
 'fechaCaptura': 4927,
 'ejercicioFiscal': 4927,
 'rfc': 4927,
 'curp': 4927,
 'nombres': 4927,
 'primerApellido': 4927,
 'segundoApellido': 4927,
 'tipoArea': 4927,
 'nivelResponsabilidad': 4927,
 'tipoProcedimiento': 4927,
 'observaciones': 1,
 'ramo.clave': 4927,
 'ramo.valor': 4927,
 'genero.clave': 4927,
 'genero.valor': 4927,
 'institucionDependencia.nombre': 4927,
 'institucionDependencia.siglas': 4927,
 'institucionDependencia.clave': 4927,
 'puesto.nombre': 1,
 'puesto.nivel': 1,
 'superiorInmediato.nombres': 4927,
 'superiorInmediato.primerApellido': 4927,
 'superiorInmediato.segundoApellido': 4927,
 'superiorInmediato.curp': 4927,
 'superiorInmediato.rfc': 4927,
 'superiorInmediato.puesto.nombre': 4927,
 'superiorInmediato.puesto.nivel': 4927,
 'puesto': 4926,
 'superiorInmediato.genero.clave': 4926,
 'superiorInmediato.genero.valor': 4926}

In [19]:
df_fun_contra['curp']

0       ROCA900530MYNDLL04
1       CÓGP760116HAGRVT08
2       CASC850118HVZRLR03
3       FOOF890119MAGNZL09
4       NAKG810014MSNVNR04
               ...        
4922    ALJG740918MSNVMR09
4923    RORL810913MPUDSR02
4924    VAMA881003HAGLDN08
4925    ARBA820123HAGMRR04
4926    VEKR741117HSNRNM06
Name: curp, Length: 4927, dtype: object

In [152]:
df_fun_contra_decl.shape

(4927, 59)

## Funcionarios sancionados

In [157]:
df_fun_sanc = pd.read_pickle(f'{dir_declaraciones}/funcionarios_sancionados_2021.pkl')\
    .rename(columns={'servidorPublicoSancionado.curp': 'curp'})
df_fun_sanc.head()

,id,fechaCaptura,expediente,autoridadSancionadora,tipoSancion,causaMotivoHechos,observaciones,institucionDependencia.nombre,servidorPublicoSancionado.rfc.rfc,servidorPublicoSancionado.rfc.homoClave,curp,servidorPublicoSancionado.nombres,servidorPublicoSancionado.primerApellido,servidorPublicoSancionado.segundoApellido,servidorPublicoSancionado.genero.clave,servidorPublicoSancionado.genero.valor,servidorPublicoSancionado.puesto,tipoFalta.clave,tipoFalta.valor,tipoFalta.descripcion,resolucion.fechaResolucion,multa.moneda.clave,multa.moneda.valor,multa.monto,inhabilitacion.plazo,inhabilitacion.fechaInicial,inhabilitacion.fechaFinal,servidorPublicoSancionado.rfc,institucion_merge
0,a94cacc9-53ba-40d1-bc04-08b765010ac0,2010-01-01,SANC/bed70faf,Contraloría Interna,"[{'clave': 'SE', 'valor': 'SANCIÓN ECONÓMICA',...",DESVÍO DE RECURSOS PÚBLICOS,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,PUDD930927,E33,PUDD930927MVZG N02,Daniela,Puga,de Anda,M,FEMENINO,Jefe de departamento,ASEX,COMETER O TOLERAR CONDUCTAS DE ACOSO SEXUAL,COMETER O TOLERAR CONDUCTAS DE ACOSO SEXUAL,2010-01-01,MXN,Peso Mexicano,13185.0,NaN,NaN,NaN,NaN,secretariadeenergia
1,6902edeb-1a0a-4ccf-af66-6b9308e0e173,2010-03-01,EXP/80ca6e1b,Contraloría Interna,"[{'clave': 'IRSC', 'valor': 'INDEMNIZACIÓN RES...",ABUSO DE FUNCIONES,Lorem Ipsum es simplemente el texto de relleno...,Comisión Federal de Electricidad,VICJ950414,E62,VICJ950414MVZLRS02,Jose Daniel,Villarreal,Carrion,M,FEMENINO,Jefe de departamento,AC,ACTO DE CORRUPCIÓN,RECIBIR SOBORNO POR UN TRÁMITE,2010-01-01,MXN,Peso Mexicano,9074.0,NaN,NaN,NaN,NaN,serviciodeinformacionagroalimentariaypesquera
2,e5f8626a-fa7f-41aa-bbe7-4685473ca62f,2010-04-01,SANC/23af791f,Contraloría Interna,"[{'clave': 'I', 'valor': 'INHABILITADO', 'desc...",TRÁFICO DE INFLUENCIAS,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,GAER730906,B58,GAER730906MVZSHC05,Ricardo,Gastélum,Echevarría,M,FEMENINO,Jefe de departamento,ENCB,ENCUBRIMIENTO,ENCUBRIMIENTO,2010-01-01,NaN,NaN,NaN,6 años,2010-01-01,2016-00-01,NaN,serviciosdealmacenamientodelnortesa
3,24700b4b-9fa1-4d46-8a84-93a53a66fe82,2010-04-31,GCPD/b0b7b091,Contraloría Interna,"[{'clave': 'M', 'valor': 'MULTADO', 'descripci...","COHECHO, ACEPTO BENEFICIO NO COMPRENDIDO EN SU...","Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,GAER730906,B58,GAER730906MVZSHC05,Ricardo,Gastélum,Echevarría,M,FEMENINO,Jefe de departamento,OTRO,OTRO,OTRO,2010-01-01,MXN,Peso Mexicano,21956.0,NaN,NaN,NaN,NaN,institutonacionaldebellasartesyliteratura
4,a89fa644-deea-49a0-883c-314cd166ceff,2010-01-01,GCPD/01a89518,Tribunal,"[{'clave': 'O', 'valor': 'OTRO', 'descripcion'...",CONTRATACIÓN INDEBIDA,"Al contrario del pensamiento popular, el texto...",Comisión Federal de Electricidad,RAQR910021,D72,RAQR910021MAGMRM06,Ramón,Ramos,Quiroz,M,FEMENINO,Jefe de departamento,IDSP,INCUMPLIMIENTO EN DECLARACION DE SITUACION PAT...,OCULTAMIENTODE PROPIEDADES,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,comisionnacionaldeacuaculturaypesca


In [156]:
df_fun_sanc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   id                                         925 non-null    object 
 1   fechaCaptura                               925 non-null    object 
 2   expediente                                 925 non-null    object 
 3   autoridadSancionadora                      925 non-null    object 
 4   tipoSancion                                925 non-null    object 
 5   causaMotivoHechos                          925 non-null    object 
 6   observaciones                              925 non-null    object 
 7   institucionDependencia.nombre              925 non-null    object 
 8   servidorPublicoSancionado.rfc.rfc          574 non-null    object 
 9   servidorPublicoSancionado.rfc.homoClave    574 non-null    object 
 10  servidorPublicoSancionado.

247

## Riesgo de contratación IMCO

In [10]:
imco = pd.read_pickle(f'{dir_declaraciones}/IRC_IMCO_2021.pkl')
imco.head()

,INSTITUCION,SIGLAS,Sector,IRC-2020,IRC-2019,IRC-2018,Progreso 19-20,Progreso 18-20,Monto-2020,Rango 2020,Monto-2019,Rango-2019,Monto-2018,Rango-2018,Diferencia Monto 18-20,Diferencia % 18-20,Diferencia Monto 19-20,Diferencia % 19-20,Riesgo Anomalías 2020,Riesgo Anomalías 2019,Riesgo Anomalías 2018,Riesgo Competencia 2020,Riesgo Competencia 2019,Riesgo Competencia 2019.1,Riesgo Transparencia 2020,Riesgo Transparencia 2019,Riesgo Transparencia 2018,institucion_merge
0,Servicio de Administración y Enajenación de Bi...,SAE,HACIENDA Y CRÉDITO PÚBLICO,NaN,NaN,18.900422,NaN,NaN,NaN,NaN,NaN,NaN,2.518243e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,NaN,20.495994,NaN,NaN,30.443397,NaN,NaN,19.779828,serviciodeadministracionyenajenaciondebienes
1,"Consejo de Promoción Turística de México, S.A....",CPTM,TURISMO,NaN,45.36129,31.813780,NaN,NaN,NaN,NaN,6590676.52,Monto bajo,1.984737e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,38.959828,27.691786,NaN,73.401238,56.095421,NaN,20.204563,23.530833,consejodepromocionturisticademexicosadecv
2,Policía Federal,PF,seguridad,NaN,NaN,40.859670,NaN,NaN,NaN,NaN,NaN,NaN,1.266091e+09,Monto alto,NaN,NaN,NaN,NaN,NaN,NaN,55.000296,NaN,NaN,55.031264,NaN,NaN,27.141366,policiafederal
3,Procuraduría General de la República,PGR,PGR,NaN,NaN,33.909026,NaN,NaN,NaN,NaN,NaN,NaN,9.511073e+08,Monto medio,NaN,NaN,NaN,NaN,NaN,NaN,26.345552,NaN,NaN,32.596300,NaN,NaN,43.187765,procuraduriageneraldelarepublica
4,Banco del Ahorro Nacional y Servicios Financie...,BANSEFI,HACIENDA Y CRÉDITO PÚBLICO,NaN,NaN,22.231333,NaN,NaN,NaN,NaN,NaN,NaN,7.508318e+08,Monto medio,NaN,NaN,NaN,NaN,NaN,NaN,33.933295,NaN,NaN,48.324705,NaN,NaN,7.656397,bancodelahorronacionalyserviciosfinancierossnc


In [32]:
df_fun_contra_imco = df_fun_contra.merge(imco, on='institucion_merge', how='left', indicator=True)
df_fun_contra_imco.head()

,id,fechaCaptura,ejercicioFiscal,rfc,curp,nombres,primerApellido,segundoApellido,tipoArea,nivelResponsabilidad,tipoProcedimiento,observaciones,ramo.clave,ramo.valor,genero.clave,genero.valor,institucionDependencia.nombre,institucionDependencia.siglas,institucionDependencia.clave,puesto.nombre,puesto.nivel,superiorInmediato.nombres,superiorInmediato.primerApellido,superiorInmediato.segundoApellido,superiorInmediato.curp,superiorInmediato.rfc,superiorInmediato.puesto.nombre,superiorInmediato.puesto.nivel,puesto,superiorInmediato.genero.clave,superiorInmediato.genero.valor,institucion_merge,INSTITUCION,SIGLAS,Sector,IRC-2020,IRC-2019,IRC-2018,Progreso 19-20,Progreso 18-20,Monto-2020,Rango 2020,Monto-2019,Rango-2019,Monto-2018,Rango-2018,Diferencia Monto 18-20,Diferencia % 18-20,Diferencia Monto 19-20,Diferencia % 19-20,Riesgo Anomalías 2020,Riesgo Anomalías 2019,Riesgo Anomalías 2018,Riesgo Competencia 2020,Riesgo Competencia 2019,Riesgo Competencia 2019.1,Riesgo Transparencia 2020,Riesgo Transparencia 2019,Riesgo Transparencia 2018,_merge
0,8a54a1ce-98b4-40f9-a52c-a272ed56f238,2018-05-21T17:32:28Z,2018,ROCA9005303Z8,ROCA900530MYNDLL04,Alejandra,Rodríguez,Calderón,"[{'clave': 'T', 'valor': 'TÉCNICA'}, {'clave':...","[{'clave': 'R', 'valor': 'RESOLUCIÓN'}]","[{'clave': 1, 'valor': ''}]",,23,Provisiones salariales y económicas,F,FEMENINO,SECRETARÍA DE ENERGÍA,SENER,XYZ987,DIRECTOR DE ADMINISTRACIÓN DE BIENES INFORMÁTICOS,M11,PEDRO,CRUZ,MEDINA,CUMP770826HMNRDD96,CUMP7708269R6,Director General,CFM2300003,NaN,NaN,NaN,secretariadeenergia,Secretaría de Energía,SENER,ENERGÍA,24.418932,15.850619,21.421646,8.568313,2.997286,4.390461e+07,Monto bajo,1.751384e+08,Monto bajo,7.357746e+07,Monto bajo,-2.967285e+07,-0.403287,-1.312338e+08,-0.749315,32.569211,26.949955,38.345705,50.200405,37.928237,50.849770,2.240067,2.344183,2.307350,both
1,1d6f5b03-0e18-4b1b-8b73-e11680a89bb8,2016-06-03T07:07:26Z,2016,CÓGP760116E91,CÓGP760116HAGRVT08,Patricio,Córdova,Guevara,"[{'clave': 'RC', 'valor': 'RESPONSABLE DE LA C...","[{'clave': 'A', 'valor': 'ATENCIÓN'}]","[{'clave': 1, 'valor': 'CONTRATACIONES PÚBLICA...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIO DE INFORMACION AGROALIMENTARIA Y PESQ...,ICIO,ERV,NaN,NaN,Safiya,MONTOUTO,TALSI,MOTS831018MDFNLF54,MOTS831018A14,Director General,K31,Jefe de Departamento,F,FEMENINO,serviciodeinformacionagroalimentariaypesquera,Servicio de Información Agroalimentaria y Pesq...,SIAP,AGRICULTURA Y DESARROLLO RURAL,28.852458,20.963096,37.772171,7.889362,-8.919713,2.005136e+08,Monto bajo,9.165187e+07,Monto bajo,4.620910e+07,Monto bajo,1.543045e+08,3.339266,1.088617e+08,1.187774,60.946951,23.398278,63.202214,33.072266,52.008649,64.356764,7.600107,4.438569,11.295957,both
2,13c81925-aa43-4c11-834c-7ceaca3f1ed1,2016-02-07T05:17:02Z,2016,CASC850118D51,CASC850118HVZRLR03,Carlos,Caraballo,Saldaña,[],"[{'clave': 'O', 'valor': 'OTRO'}, {'clave': 'A...","[{'clave': 4, 'valor': 'ASIGNACIÓN Y EMISIÓN D...",NaN,28,Participaciones a Entidades Federativas y Muni...,M,MASCULINO,SERVICIOS DE ALMACENAMIENTO DEL NORTE S.A.,ICIO,ERV,NaN,NaN,Raizy,DAGUERRE,LAVIADA,DALR971211MOXGVZ21,DALR971211A46,Jefe de departamento,O21,Secretario/a de Estado A,F,FEMENINO,serviciosdealmacenamientodelnortesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,2f52789d-31d1-4299-9fcf-19484f4af554,2021-11-21T12:22:02Z,2021,FOOF890119E26,FOOF890119MAGNZL09,Florencia,Fonseca,Orozco,[],"[{'clave': 'A', 'valor': 'ATENCIÓN'}]",[],NaN,28,Participaciones a Entidades Federativas y Muni...,F,FEMENINO,INSTITUTO NACIONAL DE BELLAS ARTES Y LITERATURA,ITUT,NST,NaN,NaN,Haydn,FORADADA,MONTESDEOCA,FOMH750903HVZRNY66,FOMH750903C34,Subdirector,N31,Director General Paraestatal A,M,MASCULINO,institutonacionaldebellasartesyliteratura,Instituto Nacional de Bellas Artes y Literatura,INBAL,cultura,38.199027,37.241643,28.803909,0.957385,9.395118,3.861989e+08,Monto bajo,7.543114e+08,Monto me

In [33]:
df_fun_contra_imco['_merge'].value_counts()

both          3421
left_only     1506
right_only       0
Name: _merge, dtype: int64

In [37]:
df_fun_contra['institucionDependencia.nombre'].nunique()

324